In [1]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import os
from scipy.cluster.vq import *
from glob import glob
from random import shuffle, seed
from sklearn import metrics
from __future__ import print_function
#import cv2
import time
import numpy as np
import theano
import theano.tensor as T
import matplotlib.cm as cm
import lasagne
from lasagne import layers
from lasagne.updates import nesterov_momentum
from nolearn.lasagne import NeuralNet
from nolearn.lasagne import visualize
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.cross_validation import train_test_split

WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.


In [10]:
def img_to_matrix(filename):
    """
    takes a filename and turns it into a numpy array of RGB pixels
    """
    img = cv2.imread(filename,0)
    #img = list(img.getdata())
    #img = map(list, img)
    img = np.array(img)
    #s = img.shape[0] * img.shape[1]
    #img_wide = img.reshape(1, s)
    return img

shirt = glob('bag/dataset/100/shirt_resized/*')
collared_tees = glob('bag/dataset/100/collared_tees_resized/*')
normal_tees = glob('bag/dataset/100/normal_tees_resized/*')

process_file = img_to_matrix

raw_data = [(process_file(filename),'shirt',filename) for filename in shirt] + \
           [(process_file(filename),'collared_tees',filename) for filename in collared_tees] + \
           [(process_file(filename),'normal_tees',filename) for filename in normal_tees]

# randomly order the data
seed(0)
shuffle(raw_data)

In [11]:
data = np.array([cd for (cd,_y,f) in raw_data])
labels = np.array([_y for (cd,_y,f) in raw_data])
y = [(0 if label == 'collared_tees' else( 1 if label=='normal_tees' else 2 )) for label in labels]

In [12]:
data = data.reshape(-1, 1, 100, 100)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=2)

In [14]:
X_train.shape

(498, 1, 100, 100)

In [15]:
np.asarray(y_train).shape

(498,)

In [ ]:
'''
    # layer maxpool2
    maxpool2_pool_size=(5, 5),
   
    # layer conv2d3
    conv2d3_num_filters=55,
    conv2d3_filter_size=(2, 2),
    conv2d3_nonlinearity=lasagne.nonlinearities.rectify,
    '''

In [ ]:
net1 = NeuralNet(
    layers=[('input', layers.InputLayer),
            ('conv2d1', layers.Conv2DLayer),
            ('maxpool1', layers.MaxPool2DLayer),
            ('conv2d2', layers.Conv2DLayer),
            #('dropout1', layers.DropoutLayer),
            ('dense', layers.DenseLayer),
            ('dropout2', layers.DropoutLayer),
            ('output', layers.DenseLayer),
            ],
    # input layer
    input_shape=(None, 1, 100, 100),
    # layer conv2d1
    conv2d1_num_filters=20,
    conv2d1_filter_size=(20, 20),
    conv2d1_nonlinearity=lasagne.nonlinearities.rectify,
    conv2d1_W=lasagne.init.GlorotUniform(),  
    # layer maxpool1
    maxpool1_pool_size=(5, 5),    
    # layer conv2d2
    conv2d2_num_filters=50,
    conv2d2_filter_size=(10, 10),
    conv2d2_nonlinearity=lasagne.nonlinearities.rectify,
    
   # dropout1
    #dropout1_p=0.5,    
    # dense
    dense_num_units=100,#300
    dense_nonlinearity=lasagne.nonlinearities.rectify, #tanh   
    # dropout2
    dropout2_p=0.5,    
    # output
    output_nonlinearity=lasagne.nonlinearities.softmax,
    output_num_units=3,
    # optimization method params
    update=nesterov_momentum,
    update_learning_rate=0.001,
    update_momentum=0.9,
    max_epochs=5,
    verbose=1,
    )

nn = net1.fit(X_train, np.asarray(y_train))

# Neural Network with 353473 learnable parameters

## Layer information

  #  name      size
---  --------  ---------
  0  input     1x100x100
  1  conv2d1   20x81x81
  2  maxpool1  20x16x16
  3  conv2d2   50x7x7
  4  dense     100
  5  dropout2  100
  6  output    3



In [12]:
'''
net1 = NeuralNet(
    layers=[('input', layers.InputLayer),
            ('conv2d1', layers.Conv2DLayer),
            ('maxpool1', layers.MaxPool2DLayer),
            ('conv2d2', layers.Conv2DLayer),
            ('maxpool2', layers.MaxPool2DLayer),
            ('conv2d3', layers.Conv2DLayer),
            #('maxpool2', layers.MaxPool2DLayer),
            ('dropout1', layers.DropoutLayer),
            ('dense', layers.DenseLayer),
            ('dropout2', layers.DropoutLayer),
            ('output', layers.DenseLayer),
            ],
    # input layer
    input_shape=(None, 1, 100, 100),
    # layer conv2d1
    conv2d1_num_filters=13,
    conv2d1_filter_size=(7, 7),
    conv2d1_nonlinearity=lasagne.nonlinearities.rectify,
    conv2d1_W=lasagne.init.GlorotUniform(),  
    # layer maxpool1
    maxpool1_pool_size=(5, 5),    
    # layer conv2d2
    conv2d2_num_filters=17,
    conv2d2_filter_size=(6, 6),
    conv2d2_nonlinearity=lasagne.nonlinearities.rectify,
    # layer maxpool2
    maxpool2_pool_size=(2, 2),
   
    # layer conv2d3
    conv2d3_num_filters=21,
    conv2d3_filter_size=(5, 5),
    conv2d3_nonlinearity=lasagne.nonlinearities.rectify,
   # dropout1
    dropout1_p=0.5,    
    # dense
    dense_num_units=100,#300
    dense_nonlinearity=lasagne.nonlinearities.rectify, #tanh   
    # dropout2
    dropout2_p=0.5,    
    # output
    output_nonlinearity=lasagne.nonlinearities.softmax,
    output_num_units=3,
    # optimization method params
    update=nesterov_momentum,
    update_learning_rate=0.001,
    update_momentum=0.9,
    max_epochs=5,
    verbose=1,
    )

nn = net1.fit(X_train, np.asarray(y_train))
'''

# Neural Network with 26372 learnable parameters

## Layer information

  #  name      size
---  --------  ---------
  0  input     1x100x100
  1  conv2d1   13x94x94
  2  maxpool1  13x18x18
  3  conv2d2   17x13x13
  4  maxpool2  17x6x6
  5  conv2d3   21x2x2
  6  dropout1  21x2x2
  7  dense     100
  8  dropout2  100
  9  output    3

  epoch    train loss    valid loss    train/val    valid acc  dur
-------  ------------  ------------  -----------  -----------  --------
      1      22.43273       1.19431     18.78298      0.28713  2558.51s
      2       1.17997       1.10732      1.06561      0.32673  2519.09s
      3       1.11394       1.09665      1.01577      0.34653  2573.15s
      4       1.10514       1.09412      1.01007      0.35644  2350.97s
      5       1.10312       1.09377      1.00854      0.31683  2490.86s


In [ ]:
import cPickle as pickle
with open('net1.pickle', 'wb') as f:
    pickle.dump(net2, f, -1)

In [ ]:
visualize.plot_conv_weights(net1.layers_['conv2d2'])

In [62]:
plt.show()

In [11]:
preds = net1.predict(X_test)

In [12]:
print (metrics.accuracy_score(np.asarray(y_test), preds))

0.328125


net1 = NeuralNet(
    layers=[('input', layers.InputLayer),
            ('conv2d1', layers.Conv2DLayer),
            ('maxpool1', layers.MaxPool2DLayer),
            ('conv2d2', layers.Conv2DLayer),
            #('maxpool2', layers.MaxPool2DLayer),
            ('dropout1', layers.DropoutLayer),
            ('dense', layers.DenseLayer),
            ('dropout2', layers.DropoutLayer),
            ('output', layers.DenseLayer),
            ],
    # input layer
    input_shape=(None, 1, 50, 50),
    # layer conv2d1
    conv2d1_num_filters=32,
    conv2d1_filter_size=(3, 3),
    conv2d1_nonlinearity=lasagne.nonlinearities.rectify,
    conv2d1_W=lasagne.init.GlorotUniform(),  
    # layer maxpool1
    maxpool1_pool_size=(2, 2),    
    # layer conv2d2
    conv2d2_num_filters=64,
    conv2d2_filter_size=(3, 3),
    conv2d2_nonlinearity=lasagne.nonlinearities.rectify,
    # layer maxpool2
    #maxpool2_pool_size=(2, 2),
   # dropout1
    dropout1_p=0.5,    
    # dense
    dense_num_units=250,
    dense_nonlinearity=lasagne.nonlinearities.rectify,    
    # dropout2
    dropout2_p=0.5,    
    # output
    output_nonlinearity=lasagne.nonlinearities.softmax,
    output_num_units=3,
    # optimization method params
    update=nesterov_momentum,
    update_learning_rate=0.01,
    update_momentum=0.9,
    max_epochs=5,
    verbose=1,
    )


nn = net1.fit(X_train, np.asarray(y_train))